In [1]:
import re
import sys
from nltk.corpus import stopwords
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Using TensorFlow backend.


In [2]:
data=pd.read_csv('Data.csv')

In [3]:
data.head()

,id,title,motion,manual_motion,govt_opp_motion,motion_party_affiliation,utt1,utt2,utt3,utt4,utt5,manual_speech,vote_speech,party_affiliation,name,rebellion
0,1,Doctor Assisted Dying,That leave be given to bring in a Bill to enab...,1,1,Lab,I trust that the House will not feel that ther...,If one considers the argument from a religious...,I accept that we shall all face death at one t...,"When we listen to my hon. Friend's case, we mu...",I do not think that we want that sort of socie...,0,0,Lab,Kevin McNamara,3.2
1,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The Secretary State talks of the lack of fuss ...,The right hon. Gentleman may not have heard fu...,North Tayside.,The Secretary of State's geography is terrible.,Would not the £40 million gap in the assumptio...,0,0,SNP,John Swinney,0.4
2,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,May I assure the right hon. Gentleman that he ...,Will the hon. Gentleman clarify what he meant ...,NaN,NaN,NaN,1,1,Lab,John McAllion,4.5
3,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,"Is the hon. Gentleman aware that, ever since t...","I rise as one who, in the past, has not been a...",It would be inappropriate to allow the debate ...,"Why are the problems of so many of our people,...",Today's debate is the first of this sort under...,0,1,Lab,Ian Davidson,3.1
4,2,Local Government Finance (Scotland),That the Local Government Finance (Scotland) O...,1,1,Lab,The hon. Gentleman will notice that the Childr...,I thank the hon. Gentleman for his generosity ...,NaN,NaN,NaN,1,1,Lab,Maria Fyfe,2.3


In [4]:
new_data=data[['id','manual_motion', 'utt1','utt2','utt3','utt4','utt5','govt_opp_motion','manual_speech','vote_speech','party_affiliation','motion_party_affiliation']]

In [5]:
new_data.describe()

,id,manual_motion,govt_opp_motion,manual_speech,vote_speech
count,1251.000000,1251.000000,1251.000000,1251.000000,1251.000000
mean,76.455635,0.396483,0.414868,0.564349,0.569944
std,33.891100,0.489362,0.492896,0.496040,0.495282
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,0.000000,0.000000,1.000000,1.000000
75%,105.000000,1.000000,1.000000,1.000000,1.000000
max,129.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
new_data["utt2"].fillna(" ", inplace=True)
new_data["utt3"].fillna(" ", inplace=True)
new_data["utt4"].fillna(" ", inplace=True)
new_data["utt5"].fillna(" ", inplace=True)

/home/adityaasinha28/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [7]:
new_data

,id,manual_motion,utt1,utt2,utt3,utt4,utt5,govt_opp_motion,manual_speech,vote_speech,party_affiliation,motion_party_affiliation
0,1,1,I trust that the House will not feel that ther...,If one considers the argument from a religious...,I accept that we shall all face death at one t...,"When we listen to my hon. Friend's case, we mu...",I do not think that we want that sort of socie...,1,0,0,Lab,Lab
1,2,1,The Secretary State talks of the lack of fuss ...,The right hon. Gentleman may not have heard fu...,North Tayside.,The Secretary of State's geography is terrible.,Would not the £40 million gap in the assumptio...,1,0,0,SNP,Lab
2,2,1,May I assure the right hon. Gentleman that he ...,Will the hon. Gentleman clarify what he meant ...,,,,1,1,1,Lab,Lab
3,2,1,"Is the hon. Gentleman aware that, ever since t...","I rise as one who, in the past, has not been a...",It would be inappropriate to allow the debate ...,"Why are the problems of so many of our people,...",Today's debate is the first of this sort under...,1,0,1,Lab,Lab
4,2,1,The hon. Gentleman will notice that the Childr...,I thank the hon. Gentleman for his generosity ...,,,,1,1,1,Lab,Lab
5,3,0,"Does my hon. Friend agree that, whereas last w...",I follow on from what my hon. Friend the Membe...,"That is correct, but PEPs were not.",There was a penalty for withdrawing from PEPs ...,"My hon. Friend makes a valid point, which I wa...",0,1,1,Con,Con
6,3,0,Does my hon. Friend agree that part of the dis...,,,,,0,1,1,Con,Con
7,3,0,"First, I must declare an interest as the chair...",I cannot understand how the new scheme will be...,I want to highlight some mechanical problems. ...,I am greatly relieved that the Government have...,The change does not meet the Government's orig...,0,1,1,Con,Con
8,4,1,My hon. Friend mentions Staffordshire and the ...,,,,,1,0,1,Lab,Lab
9,4,1,My right hon. Friend probably heard the interv...,,,,,1,0,0,Con,Lab


## Combining utterances for analysis

In [8]:
for i in new_data.index:
    #speech = new_data.at[i,'utt1']
    #processed_speech = preprocess_tweet(speech)
    new_data.at[i, 'utt1'] = new_data.at[i, 'utt1'] + " " + new_data.at[i, 'utt2'] +" " + new_data.at[i, 'utt3'] + " " +new_data.at[i, 'utt4'] + " " +new_data.at[i, 'utt5']    
    
   

In [9]:
new_data['utt1'][0]

'I trust that the House will not feel that there is a lack of compassion on any side of the argument about such a dreadful and painful subject. Many of us have had experiences similar to that of my hon. Friend the Member for Bassetlaw, either directly in our own families or on the periphery of decisions to turn off life-sustaining machines. Therefore, no one should be seen as lacking compassion. I find it strange that, having explained the present position in law, my hon. Friend suggested that legislation was required. Even in the case that he cited, the House of Lords has made the position quite clear. If a doctor administers medicine to alleviate pain and suffering and that results in the shortening of a person\'s life—although that does not necessarily happen—it is not euthanasia or murder. Much depends on the intent of the person administering the medicine. If one considers the argument from a religious point of view, different people will have different attitudes. I understand tha

In [10]:
new_data=new_data[['id','manual_motion', 'utt1','govt_opp_motion','manual_speech','vote_speech','party_affiliation','motion_party_affiliation']]

In [11]:
hansard_stopwords = ({'friend', 'honourable', 'hon', 'gentleman', 'lady','give', 'way', 'house', 'amendment', 'beg', 'move','member', 'question', 'green', 'white', 'paper', 'bill', 'statement', 'government', 'prime', 'minister','opposition', 'party', 'mr', 'rose'})
sklhansard_stopwords = text.ENGLISH_STOP_WORDS.union(hansard_stopwords)
    

In [12]:
# f= open("stopwords.txt","r+")

In [13]:
# text=f.read()

In [14]:
# text

In [15]:
# result = [x.strip() for x in text.split(',')]
# result

In [16]:
# words =  set(result)

In [17]:
# words

In [18]:
# sklhansard_stopwords.union(words)

In [19]:
def preprocess_speech(speech):
    processed_speech = []
    # Convert to lower case
    speech = speech.lower()
    # Replaces URLs with the word URL
    speech = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', speech)
    # Replace @handle with the word USER_MENTION
    speech = re.sub(r'@[\S]+', 'USER_MENTION', speech)
    # Replaces #hashtag with hashtag
    speech = re.sub(r'#(\S+)', r' \1 ', speech)
    # Remove RT (respeech)
    speech = re.sub(r'\brt\b', '', speech)
    # Replace 2+ dots with space
    speech = re.sub(r'\.{2,}', ' ', speech)
    # Strip space, " and ' from speech
    speech = speech.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    #speech = handle_emojis(speech)
    # Replace multiple spaces with a single space
    speech = re.sub(r'\s+', ' ', speech)
    words = speech.split()
    
    #f=open("stopwords.txt", 'r')
    #wordn=f.read()
    #stopwords = nltk.corpus.stopwords.words('english')
    #stopwords += re.split('; |, |\*|\n',wordn)
    
    hansard_stopwords = ({'friend', 'honourable', 'hon', 'gentleman', 'lady','give', 'way', 'house', 'amendment', 'beg', 'move','member', 'question', 'green', 'white', 'paper', 'bill', 'statement', 'government', 'prime', 'minister','opposition', 'party', 'mr', 'rose'})
    sklhansard_stopwords = text.ENGLISH_STOP_WORDS.union(hansard_stopwords)
    
    
#     f= open("stopwords.txt","r+")
#     txt=f.read()
#     result = [x.strip() for x in txt.split(',')]
#     word =  set(result)
#     sklhansard_stopwords= sklhansard_stopwords.union(word)
    
    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if word not in sklhansard_stopwords:
                   processed_speech.append(word)
                    
    print(sklhansard_stopwords)
    
    return ' '.join(processed_speech)

In [20]:
sklhansard_stopwords

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'amendment',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'beg',
           'behind',
           'being',
           'below',

In [21]:
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


In [22]:
def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

In [23]:
for i in new_data.index:
    speech = new_data.at[i,'utt1']
    processed_speech = preprocess_speech(speech)
    #print(processed_speech)
    new_data.at[i, 'utt1'] = processed_speech

frozenset({'only', 'then', 'anyhow', 'for', 'last', 'had', 'fifty', 'further', 'eleven', 'from', 'seeming', 'wherever', 'his', 'get', 'nevertheless', 'over', 'nobody', 'made', 'must', 'go', 'first', 'amongst', 'she', 'whatever', 'therein', 'per', 'at', 'down', 'around', 'take', 'no', 'amendment', 'besides', 'mine', 'do', 'anyone', 'these', 'toward', 'those', 'done', 'sincere', 'fill', 'however', 'moreover', 'side', 'anywhere', 'fire', 'who', 'in', 'so', 'co', 'thus', 'none', 'six', 'can', 'though', 'together', 'upon', 'across', 'becomes', 'via', 'question', 'serious', 'mr', 'whereas', 'afterwards', 'due', 'bottom', 'sometime', 'con', 'name', 'will', 'ie', 'else', 'him', 'indeed', 'bill', 'my', 'other', 'yourself', 'move', 'each', 'thereafter', 'myself', 'being', 'has', 'what', 'whose', 'yours', 'two', 'everyone', 'fifteen', 'mostly', 'became', 'part', 'were', 'than', 'whether', 'while', 'whither', 'herein', 'find', 'not', 'why', 'rose', 'be', 'twenty', 'it', 'eg', 'off', 're', 'much', 

frozenset({'only', 'then', 'anyhow', 'for', 'last', 'had', 'fifty', 'further', 'eleven', 'from', 'seeming', 'wherever', 'his', 'get', 'nevertheless', 'over', 'nobody', 'made', 'must', 'go', 'first', 'amongst', 'she', 'whatever', 'therein', 'per', 'at', 'down', 'around', 'take', 'no', 'amendment', 'besides', 'mine', 'do', 'anyone', 'these', 'toward', 'those', 'done', 'sincere', 'fill', 'however', 'moreover', 'side', 'anywhere', 'fire', 'who', 'in', 'so', 'co', 'thus', 'none', 'six', 'can', 'though', 'together', 'upon', 'across', 'becomes', 'via', 'question', 'serious', 'mr', 'whereas', 'afterwards', 'due', 'bottom', 'sometime', 'con', 'name', 'will', 'ie', 'else', 'him', 'indeed', 'bill', 'my', 'other', 'yourself', 'move', 'each', 'thereafter', 'myself', 'being', 'has', 'what', 'whose', 'yours', 'two', 'everyone', 'fifteen', 'mostly', 'became', 'part', 'were', 'than', 'whether', 'while', 'whither', 'herein', 'find', 'not', 'why', 'rose', 'be', 'twenty', 'it', 'eg', 'off', 're', 'much', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
new_data['utt1']

0       trust feel lack compassion argument dreadful p...
1       secretary state talks lack fuss local settleme...
2       assure right probably thinking sad matters lon...
3       aware beginning urban aid programme projects l...
4       notice children scotland act enacted operation...
5       does agree week treated peter pan economics ca...
6       does agree disaster governments plans announce...
7       declare chairman investment management busines...
8       mentions staffordshire birmingham northern rel...
9       right probably heard intervention hayes harlin...
10      does right agree issue just numbers abandonmen...
11      aware gloucestershire police lowest increase c...
12      welcome right aware representations oldham met...
13      right aware public claim ruth coleman liberal ...
14      grateful secretary state giving impressed rema...
15      point local communities supporting council tax...
16      barnsley council year received highest settlem...
17      having

In [25]:
#max_features = 100000
# tokenizer = Tokenizer(num_words=max_features, split=' ')
# tokenizer.fit_on_texts(new_data['utt1'].values)
# X = tokenizer.texts_to_sequences(new_data['utt1'].values)
# X = pad_sequences(X)

In [26]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# #hansard_stopwords = ({'friend', 'honourable', 'hon', 'gentleman', 'lady','give', 'way', 'house', 'amendment', 'beg', 'move','member', 'question', 'green', 'white', 'paper', 'bill', 'statement', 'government', 'prime', 'minister','opposition', 'party', 'mr', 'rose'})
# #sklhansard_stopwords = text.ENGLISH_STOP_WORDS.union(hansard_stopwords)
# vectorizer = TfidfVectorizer(min_df=5, max_df = 1.0, 
# ngram_range=(1,3), sublinear_tf=True, use_idf =True, 
# stop_words=sklhansard_stopwords)

In [27]:
val=new_data['utt1']
val.shape

(1251,)

In [28]:
Y=new_data['manual_speech']
Y

0       0
1       0
2       1
3       0
4       1
5       1
6       1
7       1
8       0
9       0
10      0
11      0
12      1
13      0
14      0
15      1
16      1
17      1
18      0
19      0
20      1
21      0
22      1
23      1
24      1
25      1
26      1
27      1
28      0
29      1
       ..
1221    1
1222    1
1223    1
1224    1
1225    0
1226    0
1227    0
1228    1
1229    1
1230    0
1231    0
1232    0
1233    0
1234    1
1235    0
1236    1
1237    1
1238    1
1239    0
1240    0
1241    1
1242    1
1243    1
1244    1
1245    1
1246    0
1247    1
1248    1
1249    1
1250    0
Name: manual_speech, Length: 1251, dtype: int64

In [29]:
# num_data=new_data[['govt_opp_motion','id','party_code','vote_speech','motion_party_code']]

In [30]:
# X_train, X_val, y_train, y_val = train_test_split(val,Y, test_size=0.2, random_state=123 )

In [31]:
#all_words=' '.join(val)
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=''
for i in range(len(val)):
    all_words=all_words + val[i]

all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

11291

In [32]:
r_len=[]
for text in val:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)

max_review_len= np.max(r_len)
max_review_len

423

In [33]:
max_features=num_unique_word
max_words=max_review_len

In [367]:
tokenizer = Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(val)

In [368]:
val = tokenizer.texts_to_sequences(val)
val = pad_sequences(val, max_words)

In [369]:
val.shape

(1251, 423)

In [370]:
 word_index = tokenizer.word_index

In [371]:
len(word_index)

10167

In [34]:
path='glove.6B.100d.txt'

In [35]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
 

In [36]:
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix

In [375]:
# EMBEDDING_FILE = 'glove.6B.100d.txt'
# embed_dim = 100 #word vector dim
# embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
# print(embedding_matrix.shape)

In [376]:
# embeddings_index = {}
# with open(path) as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:], dtype='utf-8')
#         embeddings_index[word] = coefs

In [37]:
Y=new_data['manual_speech']
Y

0       0
1       0
2       1
3       0
4       1
5       1
6       1
7       1
8       0
9       0
10      0
11      0
12      1
13      0
14      0
15      1
16      1
17      1
18      0
19      0
20      1
21      0
22      1
23      1
24      1
25      1
26      1
27      1
28      0
29      1
       ..
1221    1
1222    1
1223    1
1224    1
1225    0
1226    0
1227    0
1228    1
1229    1
1230    0
1231    0
1232    0
1233    0
1234    1
1235    0
1236    1
1237    1
1238    1
1239    0
1240    0
1241    1
1242    1
1243    1
1244    1
1245    1
1246    0
1247    1
1248    1
1249    1
1250    0
Name: manual_speech, Length: 1251, dtype: int64

In [38]:
# val_train=vectorizer.fit_transform(val.tolist())
# val_train

## Using a TF-IDF Vectorizer and Transformer

In [39]:
# from sklearn.feature_extraction.text import TfidfTransformer
# transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
# # speeches_train_corpus = transformer.fit_transform(speeches_train_corpus)
# # speeches_test_corpus = transformer.transform(speeches_test_corpus)
# val_train = transformer.fit_transform(val_train)

In [40]:
val.shape
val_train=val
val_train

0       trust feel lack compassion argument dreadful p...
1       secretary state talks lack fuss local settleme...
2       assure right probably thinking sad matters lon...
3       aware beginning urban aid programme projects l...
4       notice children scotland act enacted operation...
5       does agree week treated peter pan economics ca...
6       does agree disaster governments plans announce...
7       declare chairman investment management busines...
8       mentions staffordshire birmingham northern rel...
9       right probably heard intervention hayes harlin...
10      does right agree issue just numbers abandonmen...
11      aware gloucestershire police lowest increase c...
12      welcome right aware representations oldham met...
13      right aware public claim ruth coleman liberal ...
14      grateful secretary state giving impressed rema...
15      point local communities supporting council tax...
16      barnsley council year received highest settlem...
17      having

In [41]:
val_train.shape

(1251,)

In [42]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
new_data["party_code"] = lb_make.fit_transform(new_data["party_affiliation"])
new_data

lb_mp = LabelEncoder()
new_data["motion_party_code"] = lb_mp.fit_transform(new_data["motion_party_affiliation"])
new_data



/home/adityaasinha28/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/adityaasinha28/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,manual_motion,utt1,govt_opp_motion,manual_speech,vote_speech,party_affiliation,motion_party_affiliation,party_code,motion_party_code
0,1,1,trust feel lack compassion argument dreadful p...,1,0,0,Lab,Lab,8,3
1,2,1,secretary state talks lack fuss local settleme...,1,0,0,SNP,Lab,11,3
2,2,1,assure right probably thinking sad matters lon...,1,1,1,Lab,Lab,8,3
3,2,1,aware beginning urban aid programme projects l...,1,0,1,Lab,Lab,8,3
4,2,1,notice children scotland act enacted operation...,1,1,1,Lab,Lab,8,3
5,3,0,does agree week treated peter pan economics ca...,0,1,1,Con,Con,0,0
6,3,0,does agree disaster governments plans announce...,0,1,1,Con,Con,0,0
7,3,0,declare chairman investment management busines...,0,1,1,Con,Con,0,0
8,4,1,mentions staffordshire birmingham northern rel...,1,0,1,Lab,Lab,8,3
9,4,1,right probably heard intervention hayes harlin...,1,0,0,Con,Lab,0,3


In [43]:
#num_data=new_data[['manual_motion','govt_opp_motion','vote_speech','id']]
num_data=new_data[['govt_opp_motion','id','party_code','motion_party_code','manual_motion']]
num_data

,govt_opp_motion,id,party_code,motion_party_code,manual_motion
0,1,1,8,3,1
1,1,2,11,3,1
2,1,2,8,3,1
3,1,2,8,3,1
4,1,2,8,3,1
5,0,3,0,0,0
6,0,3,0,0,0
7,0,3,0,0,0
8,1,4,8,3,1
9,1,4,0,3,1


# Combining metadata with the text for prediction
"Manual_Speech" is predicted by training on "text", "govt_opp_motion", "id", "manual_motion", "party_code", "vote_speech" and "motion_party_code"

In [384]:
from scipy.sparse import hstack as hs
from numpy import hstack

In [385]:
num_data.values

array([[  1,   1,   8,   3,   1],
       [  1,   2,  11,   3,   1],
       [  1,   2,   8,   3,   1],
       ...,
       [  0, 129,   1,   4,   0],
       [  0, 129,  11,   4,   0],
       [  0, 129,   0,   4,   0]])

In [386]:
num_data.shape

(1251, 5)

In [387]:
val.shape

(1251, 423)

In [388]:
from scipy.sparse import hstack as hs
from numpy import hstack


In [389]:
new_train_data=hs([val,num_data])

In [390]:
new_train_data.shape

(1251, 428)

In [391]:
# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(list(val_train))
# X_train = tokenizer.texts_to_sequences(val_train)
# X_test = tokenizer.texts_to_sequences(X_test)

In [392]:
# new_train_data=hs([val_train,num_data])

In [393]:
val_train.shape

(1251, 423)

In [394]:
# X_train, X_test, Y_train, Y_test = train_test_split(val_train,Y, test_size = 0.2, random_state = 123)
# X_train

In [395]:
# arr=np.array(X_train.index)

In [396]:
# arr

In [397]:
# from nltk.tokenize import word_tokenize
# from nltk import FreqDist
# from nltk.stem import SnowballStemmer,WordNetLemmatizer

# all_words=' '.join(X_train)
# all_words=word_tokenize(all_words)
# dist=FreqDist(all_words)
# num_unique_word=len(dist)
# num_unique_word

In [398]:
# r_len=[]
# for text in X_train:
#     word=word_tokenize(text)
#     l=len(word)
#     r_len.append(l)
    
# MAX_REVIEW_LEN=np.max(r_len)
# MAX_REVIEW_LEN

# # MIN_REVIEW_LEN=np.min(r_len)
# # MIN_REVIEW_LEN

In [47]:
max_features = num_unique_word + 5
#max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 8
num_classes=1

In [400]:
max_features

11296

In [401]:
# embedding matrix
EMBEDDING_FILE = 'glove.6B.100d.txt'
embed_dim = 100 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)

Found 400000 word vectors.
(10168, 100)


In [402]:
# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(list(X_train))
# X_train = tokenizer.texts_to_sequences(X_train)
# X_test = tokenizer.texts_to_sequences(X_test)

In [403]:
X_train, X_test, y_train, y_test = train_test_split(new_train_data,Y, test_size=0.20, random_state=123 )

In [404]:
# from keras.preprocessing import sequence,text
# from keras.preprocessing.text import Tokenizer

# X_train = sequence.pad_sequences(X_train, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)
# print(X_train.shape,X_test.shape)

In [405]:
X_train.shape

(1000, 428)

In [406]:
embed_dim

100

In [407]:
# embedding matrix
EMBEDDING_FILE = 'glove.6B.100d.txt'
embed_dim = 100 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)

Found 400000 word vectors.
(10168, 100)


In [408]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

In [417]:
model5 = Sequential()
model5.add(Embedding(len(word_index)+1, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model5.add(SpatialDropout1D(0.25))
model5.add(Bidirectional(GRU(256,return_sequences=True)))
model5.add(Bidirectional(GRU(128,return_sequences=True)))
model5.add(Bidirectional(GRU(64,return_sequences=False)))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='sigmoid'))
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 428, 100)          1016800   
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, 428, 100)          0         
_________________________________________________________________
bidirectional_24 (Bidirectio (None, 428, 512)          548352    
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 428, 256)          492288    
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 128)               123264    
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 129       
Total para

In [418]:
model5.fit(X_train, y_train, epochs = 8, batch_size=128, verbose = 1)

Epoch 1/8
1000/1000 [==============================] - 95s 95ms/step - loss: 0.7204 - acc: 0.5330
Epoch 2/8
1000/1000 [==============================] - 91s 91ms/step - loss: 0.6507 - acc: 0.6150
Epoch 3/8
1000/1000 [==============================] - 92s 92ms/step - loss: 0.5774 - acc: 0.7050
Epoch 4/8
1000/1000 [==============================] - 92s 92ms/step - loss: 0.5267 - acc: 0.7300
Epoch 5/8
1000/1000 [==============================] - 92s 92ms/step - loss: 0.4764 - acc: 0.7580
Epoch 6/8
1000/1000 [==============================] - 93s 93ms/step - loss: 0.4298 - acc: 0.8110
Epoch 7/8
1000/1000 [==============================] - 93s 93ms/step - loss: 0.4062 - acc: 0.8060
Epoch 8/8
1000/1000 [==============================] - 92s 92ms/step - loss: 0.3475 - acc: 0.8530


In [419]:
scores = model5.evaluate(X_test, y_test, verbose=1)
print("%s: %.2f%%" % (model5.metrics_names[1], scores[1]*100))

251/251 [==============================] - 10s 38ms/step
acc: 89.24%


In [420]:
preds=model5.predict(X_test)

In [421]:
preds = preds.round()

In [422]:
preds

array([[0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],

In [423]:
val = f1_score(y_test, preds, average='macro') 

In [424]:
val

0.8914377252703245

In [317]:
val

0.8859683281412254

In [361]:
val

0.908423100489185

In [203]:
model1=Sequential()
model1.add(Embedding(len(word_index)+1, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 398, 100)          964100    
_________________________________________________________________
lstm_1 (LSTM)                (None, 398, 64)           42240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 1,018,789
Trainable params: 1,018,789
Non-trainable params: 0
_________________________________________________________________


In [204]:
model1.fit(X_train, y_train, epochs = 8, batch_size=128, verbose = 1)

Epoch 1/8
1000/1000 [==============================] - 9s 9ms/step - loss: 0.6845 - acc: 0.5450
Epoch 2/8
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6793 - acc: 0.5640
Epoch 3/8
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6752 - acc: 0.5890
Epoch 4/8
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6662 - acc: 0.6040
Epoch 5/8
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6570 - acc: 0.5990
Epoch 6/8
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6463 - acc: 0.6150
Epoch 7/8
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6203 - acc: 0.6640
Epoch 8/8
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6131 - acc: 0.6750


In [205]:
scores = model1.evaluate(X_test, y_test, verbose=1)
print("%s: %.2f%%" % (model1.metrics_names[1], scores[1]*100))

251/251 [==============================] - 2s 6ms/step
acc: 70.12%


In [207]:
f1_score(y_test, (model1.predict(X_test)).round(), average='weighted') 

0.6930461350168424

In [212]:
model2= Sequential()
model2.add(Embedding(len(word_index)+1, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='sigmoid'))


model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 398, 100)          964100    
_________________________________________________________________
dropout_8 (Dropout)          (None, 398, 100)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 398, 64)           19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total para

In [213]:
model2.fit(X_train, y_train, epochs = 16, batch_size=64, verbose = 1)

Epoch 1/16
1000/1000 [==============================] - 4s 4ms/step - loss: 0.7818 - acc: 0.4950
Epoch 2/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6905 - acc: 0.5560
Epoch 3/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6571 - acc: 0.6080
Epoch 4/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6101 - acc: 0.6630
Epoch 5/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5666 - acc: 0.7040
Epoch 6/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5037 - acc: 0.7700
Epoch 7/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4329 - acc: 0.8240
Epoch 8/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3862 - acc: 0.8330
Epoch 9/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3326 - acc: 0.8680
Epoch 10/16
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2777 - acc: 0.9060
Epoch 11/16
1000/1000 [======

In [215]:
val = f1_score(y_test, (model2.predict(X_test)).round(), average='micro') 

In [216]:
val

0.8286852589641436

In [217]:
model3= Sequential()
model3.add(Embedding(len(word_index)+1, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 398, 100)          964100    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 398, 64)           19264     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 199, 64)           0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 199, 64)           0         
_________________________________________________________________
gru_10 (GRU)                 (None, 199, 128)          74112     
_________________________________________________________________
dropout_11 (Dropout)         (None, 199, 128)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25472)             0         
__________

In [218]:
model3.fit(X_train, y_train, epochs = 16, batch_size=64, verbose = 1)

Epoch 1/16
1000/1000 [==============================] - 9s 9ms/step - loss: 1.0251 - acc: 0.5350
Epoch 2/16
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6919 - acc: 0.5540
Epoch 3/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6670 - acc: 0.5990
Epoch 4/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6523 - acc: 0.6600
Epoch 5/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.5338 - acc: 0.7450
Epoch 6/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.4144 - acc: 0.8220
Epoch 7/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3393 - acc: 0.8590
Epoch 8/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2620 - acc: 0.9030
Epoch 9/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2265 - acc: 0.9140
Epoch 10/16
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1764 - acc: 0.9390
Epoch 11/16
1000/1000 [======

In [219]:
f1_score(y_test, (model3.predict(X_test)).round(), average='weighted') 

0.7544618517376424

In [221]:
model4 = Sequential()

model4.add(Embedding(len(word_index)+1, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 398, 100)          964100    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 398, 100)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 256)               175872    
_________________________________________________________________
dropout_13 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 1,140,229
Trainable params: 1,140,229
Non-trainable params: 0
_________________________________________________________________


In [223]:
model4.fit(X_train, y_train, epochs = 16, batch_size=64, verbose = 1)

Epoch 1/16
1000/1000 [==============================] - 16s 16ms/step - loss: 0.6791 - acc: 0.5770
Epoch 2/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5798 - acc: 0.6940
Epoch 3/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5054 - acc: 0.7340
Epoch 4/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4141 - acc: 0.8030
Epoch 5/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3647 - acc: 0.8430
Epoch 6/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3384 - acc: 0.8540
Epoch 7/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3129 - acc: 0.8760
Epoch 8/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2707 - acc: 0.8890
Epoch 9/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2131 - acc: 0.9200
Epoch 10/16
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1913 - acc: 0.9280
Epoch 11/

In [114]:
model4.fit(X_train, Y_train, epochs = 8, batch_size=64, verbose = 1)

Epoch 1/8
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0690 - acc: 0.9780
Epoch 2/8
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0680 - acc: 0.9760
Epoch 3/8
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0572 - acc: 0.9760
Epoch 4/8
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0497 - acc: 0.9830
Epoch 5/8
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0468 - acc: 0.9860
Epoch 6/8
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0321 - acc: 0.9880
Epoch 7/8
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0326 - acc: 0.9890
Epoch 8/8
1000/1000 [==============================] - 14s 14ms/step - loss: 0.0270 - acc: 0.9950


In [224]:
f1_score(y_test, (model4.predict(X_test)).round(), average='micro') 

0.8247011952191236

In [117]:
model6 = Sequential()
model6.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model6.add(SpatialDropout1D(0.25))
model6.add(Bidirectional(LSTM(128,return_sequences=True)))
model6.add(Bidirectional(LSTM(64,return_sequences=False)))
model6.add(Dropout(0.5))
model6.add(Dense(num_classes, activation='sigmoid'))
model6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 394, 100)          849600    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 394, 100)          0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 394, 256)          234496    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               164352    
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 1,248,577
Trainable params: 1,248,577
Non-trainable params: 0
_________________________________________________________________


In [74]:
def create_LSTM():
    embed_dim = 128
    lstm_out = 196
    max_features=200
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
    
    model5 = Sequential()
    model5.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
    model5.add(SpatialDropout1D(0.25))
    model5.add(Bidirectional(GRU(128,return_sequences=True)))
    model5.add(Bidirectional(GRU(64,return_sequences=False)))
    model5.add(Dropout(0.5))
    model5.add(Dense(num_classes, activation='softmax'))
    model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    
    return model

In [118]:
model6.fit(X_train, Y_train, epochs = 16, batch_size=64, verbose = 1)

Epoch 1/16
1000/1000 [==============================] - 39s 39ms/step - loss: 0.6988 - acc: 0.5430
Epoch 2/16
1000/1000 [==============================] - 36s 36ms/step - loss: 0.6915 - acc: 0.5520
Epoch 3/16
1000/1000 [==============================] - 36s 36ms/step - loss: 0.6741 - acc: 0.5800
Epoch 4/16
1000/1000 [==============================] - 36s 36ms/step - loss: 0.6516 - acc: 0.6210
Epoch 5/16
1000/1000 [==============================] - 36s 36ms/step - loss: 0.6306 - acc: 0.6640
Epoch 6/16
1000/1000 [==============================] - 36s 36ms/step - loss: 0.5871 - acc: 0.6970
Epoch 7/16
1000/1000 [==============================] - 36s 36ms/step - loss: 0.5654 - acc: 0.7030
Epoch 8/16
1000/1000 [==============================] - 36s 36ms/step - loss: 0.4911 - acc: 0.7750
Epoch 9/16
1000/1000 [==============================] - 36s 36ms/step - loss: 0.4569 - acc: 0.8000
Epoch 10/16
1000/1000 [==============================] - 38s 38ms/step - loss: 0.3474 - acc: 0.8530
Epoch 11/

In [121]:
f1_score(Y_test, (model6.predict(X_test)).round(), average='weighted')

0.5598124426983747

In [226]:
from keras.layers import (Input, Dense, Embedding, LSTM, GRU, Bidirectional, 
                        SpatialDropout1D,  GlobalMaxPooling1D, Concatenate, 
                        Conv1D, Dropout, BatchNormalization, Activation)
from keras.models import Model

In [229]:
inp = Input((X_train.shape[1],))
#x= Embedding(len(word_index)+1, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True)
x =  Embedding(max_features, embed_dim,input_length = X_train.shape[1])(inp)
x1 = x
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(256, return_sequences=True))(x)
x = Conv1D(256, 7, strides=2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
    
x2 = x
x = SpatialDropout1D(0.)(x)
x = Bidirectional(GRU(128, return_sequences=True))(x)
x = Conv1D(128, 7, strides=2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
    
x3 = x
x = SpatialDropout1D(0.)(x)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = Conv1D(64, 7, strides=2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
    
x_o1 = GlobalMaxPooling1D()(x)
x_o2 = Bidirectional(GRU(embed_dim, return_sequences=False))(x1)
x_o3 = Bidirectional(GRU(256, return_sequences=False))(x2)
x_o4 = Bidirectional(GRU(128, return_sequences=False))(x3)
x_o5 = Bidirectional(GRU(64, return_sequences=False))(x)
    
x = Concatenate()([x_o1, x_o2, x_o3, x_o4, x_o5])
x = Dense(128, activation='tanh')(x)
out = Dense(1, activation='sigmoid')(x)
    
model = Model(inputs=inp, outputs=out)

In [230]:
from keras.optimizers import Adam, SGD, Nadam
from keras.models import Model
from keras import backend as K

In [231]:
opt = Nadam(lr=1e-3, schedule_decay=0.04)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [233]:
model.fit(X_train, y_train, epochs = 16, batch_size=64, verbose = 1)

Epoch 1/16
1000/1000 [==============================] - 119s 119ms/step - loss: 0.9237 - acc: 0.5400
Epoch 2/16
1000/1000 [==============================] - 110s 110ms/step - loss: 0.6623 - acc: 0.6290
Epoch 3/16
1000/1000 [==============================] - 112s 112ms/step - loss: 0.5127 - acc: 0.7550
Epoch 4/16
1000/1000 [==============================] - 112s 112ms/step - loss: 0.3123 - acc: 0.8630
Epoch 5/16
1000/1000 [==============================] - 118s 118ms/step - loss: 0.2582 - acc: 0.9010
Epoch 6/16
1000/1000 [==============================] - 126s 126ms/step - loss: 0.0340 - acc: 0.9920
Epoch 7/16
1000/1000 [==============================] - 126s 126ms/step - loss: 0.0148 - acc: 0.9980
Epoch 8/16
1000/1000 [==============================] - 115s 115ms/step - loss: 0.0102 - acc: 0.9980
Epoch 9/16
1000/1000 [==============================] - 120s 120ms/step - loss: 0.0053 - acc: 0.9980
Epoch 10/16
1000/1000 [==============================] - 116s 116ms/step - loss: 0.0029 - a

In [235]:
f1_score(y_test, (model.predict(X_test)).round(), average='weighted')

0.7711084426724446

In [51]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.layers import Dense, Dropout, Activation , GRU
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D, Bidirectional
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM, Convolution1D, Flatten, Dropout, Dense
folds_no = 10
kf = StratifiedKFold(n_splits=folds_no, shuffle=True)

In [45]:
num_data_arr=np.array(num_data)
num_data.iloc[[1,2]]

,govt_opp_motion,id,party_code,motion_party_code,manual_motion
1,1,2,11,3,1
2,1,2,8,3,1


In [46]:
val

0       trust feel lack compassion argument dreadful p...
1       secretary state talks lack fuss local settleme...
2       assure right probably thinking sad matters lon...
3       aware beginning urban aid programme projects l...
4       notice children scotland act enacted operation...
5       does agree week treated peter pan economics ca...
6       does agree disaster governments plans announce...
7       declare chairman investment management busines...
8       mentions staffordshire birmingham northern rel...
9       right probably heard intervention hayes harlin...
10      does right agree issue just numbers abandonmen...
11      aware gloucestershire police lowest increase c...
12      welcome right aware representations oldham met...
13      right aware public claim ruth coleman liberal ...
14      grateful secretary state giving impressed rema...
15      point local communities supporting council tax...
16      barnsley council year received highest settlem...
17      having

## Running the LSTM model with 10-fold cross validation 

In [52]:
from scipy.sparse import hstack as hs
from numpy import hstack

cvscores = []
#X_sparse=new_train_data.todense()
for train_index, test_index in kf.split(val,Y):
        
    
        
    X_train, X_test = val[train_index], val[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    num_data_train, num_data_test = num_data.iloc[train_index],num_data.iloc[test_index]
   
    from nltk.tokenize import word_tokenize
    from nltk import FreqDist
    
#     all_words=''
#     for i in range(len(X_train)):
#         all_words=all_words + val[i]

#     all_words=word_tokenize(all_words)
#     dist=FreqDist(all_words)
#     num_unique_word=len(dist)
#     #num_unique_word=9641
    
#     r_len=[]
#     for text in X_train:
#         word=word_tokenize(text)
#         l=len(word)
#         r_len.append(l)

#     max_review_len= np.max(r_len)

    #max_review_len = 394
    
#     max_features=num_unique_word
#     max_words=max_review_len
    
    #print("Max features = ", max_features)
    
    tokenizer = Tokenizer(num_words=max_features, filters='')
    tokenizer.fit_on_texts(X_train)
    
    X_train = tokenizer.texts_to_sequences(X_train)
    X_train = pad_sequences(X_train, max_words)
    
    X_test = tokenizer.texts_to_sequences(X_test)
    X_test = pad_sequences(X_test, max_words)
    
    
    word_index = tokenizer.word_index
    #X_val_train=vectorizer.fit_transform(X_train.tolist())
    #X_val_test=vectorizer.transform(X_test.tolist())
    
    X_train = hs([X_train, num_data_train])
    X_test = hs([X_test, num_data_test])
    
    
    EMBEDDING_FILE = 'glove.6B.100d.txt'
    embed_dim = 100 #word vector dim
    embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)

    #transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
    
    #print("Embed matrix shape", embedding_matrix.shape)
    #X_val_train = transformer.fit_transform(X_val_train)
    #X_val_test=transformer.transform(X_val_test)
    
    #X_new_train_data=hs([X_train,num_data_train])
    #X_new_test_data=hs([X_test,num_data_test])
    
    #embed_dim = 100
    #lstm_out = 196
    #max_features=100
    #model = Sequential()
    #model.add(Embedding(1000, embed_dim,input_length = (X_new_train_data.toarray()).shape[1]))
    #model.add(SpatialDropout1D(0.4))
    #model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
    #model.add(Dense(1,activation='sigmoid'))
    #model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

    model = Sequential()
    #model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
    #model.add(Embedding(embedding_matrix.shape[1], embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
    model.add(Embedding(len(word_index)+1, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
    model.add(SpatialDropout1D(0.25))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(Bidirectional(GRU(128,return_sequences=True)))
    model.add(Bidirectional(GRU(64,return_sequences=False)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #model.fit(X_new_train_data.toarray(), y_train, epochs=10, batch_size=32, verbose=1)
    # evaluate the model
    
    model.fit(X_train.toarray(), y_train, epochs=10, batch_size=128, verbose=1)
    scores = model.evaluate(X_test.toarray(), y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    f1scor=f1_score(y_test, (model.predict(X_test.toarray())).round(), average='weighted')
    #print("Values are" ,model.metrics_names, scores)
    print("f2 score is", f1scor)
    
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



Found 400000 word vectors.
Epoch 1/10
1125/1125 [==============================] - 108s 96ms/step - loss: 0.7423 - acc: 0.5627
Epoch 2/10
1125/1125 [==============================] - 105s 93ms/step - loss: 0.6378 - acc: 0.6356
Epoch 3/10
1125/1125 [==============================] - 104s 93ms/step - loss: 0.5526 - acc: 0.7013
Epoch 4/10
1125/1125 [==============================] - 105s 94ms/step - loss: 0.4972 - acc: 0.7476
Epoch 5/10
1125/1125 [==============================] - 105s 94ms/step - loss: 0.4670 - acc: 0.7858
Epoch 6/10
1125/1125 [==============================] - 105s 94ms/step - loss: 0.4184 - acc: 0.8133
Epoch 7/10
1125/1125 [==============================] - 106s 94ms/step - loss: 0.4010 - acc: 0.8151
Epoch 8/10
1125/1125 [==============================] - 106s 94ms/step - loss: 0.3702 - acc: 0.8409
Epoch 9/10
1125/1125 [==============================] - 105s 93ms/step - loss: 0.3180 - acc: 0.8622
Epoch 10/10
1125/1125 [==============================] - 108s 96ms/step -

1127/1127 [==============================] - 103s 91ms/step - loss: 0.3702 - acc: 0.8367
Epoch 8/10
1127/1127 [==============================] - 104s 93ms/step - loss: 0.3511 - acc: 0.8447
Epoch 9/10
1127/1127 [==============================] - 103s 91ms/step - loss: 0.3182 - acc: 0.8713
Epoch 10/10
1127/1127 [==============================] - 106s 94ms/step - loss: 0.3045 - acc: 0.8793
acc: 79.84%
f2 score is 0.7953118541990593
Found 400000 word vectors.
Epoch 1/10
1127/1127 [==============================] - 107s 95ms/step - loss: 0.7403 - acc: 0.5457
Epoch 2/10
1127/1127 [==============================] - 103s 92ms/step - loss: 0.6253 - acc: 0.6424
Epoch 3/10
1127/1127 [==============================] - 102s 90ms/step - loss: 0.5507 - acc: 0.7090
Epoch 4/10
1127/1127 [==============================] - 103s 92ms/step - loss: 0.4884 - acc: 0.7764
Epoch 5/10
1127/1127 [==============================] - 103s 92ms/step - loss: 0.4322 - acc: 0.7977
Epoch 6/10
1127/1127 [==================

In [54]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

85.12% (+/- 3.40%)


In [56]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

85.12% (+/- 3.40%)


In [57]:
cvscores

[90.47619057080102,
 85.71428505201189,
 86.50793603488377,
 81.74603108375791,
 88.09523828445919,
 87.20000052452087,
 83.06451612903226,
 79.83870910060021,
 87.90322638327076,
 80.64516090577648]

In [57]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

91.77% (+/- 1.92%)
